In [32]:
import pandas as pd
import numpy as np

## 1.1

Importación de archivos

In [15]:

data_train = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gold_recovery_train.csv")
data_test = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gold_recovery_test.csv")
data_full = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gold_recovery_full.csv")

## 1.2

Comprueba que el calculo de recuperación sea correcto

In [ ]:
#Columna con calculo de recuperación: data_train["rougher.output.recovery"]

# Función que calcula el % de recuperación
def recovery(df):
    c = df["rougher.output.concentrate_au"]
    f = df["rougher.input.feed_au"]
    t = df["rougher.output.tail_au"]
    try:
        result = ((c*(f-t))/(f*(c-t)))*100
    except:
        result = 0
    return result

recovery_calc = data_train.apply(lambda x: recovery(x), axis = 1)

print("Error absoluto medio del cálculo de recuperación:", (recovery_calc - data_train["rougher.output.recovery"]).abs().mean())

Error absoluto medio del cálculo de recuperación: 9.303415616264301e-15


Al tener un error absoluto medio tan bajo, podemos afirmar que los calculos han sido correctos hasta el momento

## 1.3

Analiza las características no disponibles del conjunto de prueba

In [22]:
not_in_test = set(data_train.columns) - set(data_test.columns)
not_in_test

{'final.output.concentrate_ag',
 'final.output.concentrate_au',
 'final.output.concentrate_pb',
 'final.output.concentrate_sol',
 'final.output.recovery',
 'final.output.tail_ag',
 'final.output.tail_au',
 'final.output.tail_pb',
 'final.output.tail_sol',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_au',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_sol',
 'primary_cleaner.output.tail_ag',
 'primary_cleaner.output.tail_au',
 'primary_cleaner.output.tail_pb',
 'primary_cleaner.output.tail_sol',
 'rougher.calculation.au_pb_ratio',
 'rougher.calculation.floatbank10_sulfate_to_au_feed',
 'rougher.calculation.floatbank11_sulfate_to_au_feed',
 'rougher.calculation.sulfate_to_au_concentrate',
 'rougher.output.concentrate_ag',
 'rougher.output.concentrate_au',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_sol',
 'rougher.output.recovery',
 'rougher.output.tail_ag',
 'rougher.output.tail_au',
 'rougher.output.ta

Podemos observar que las columnas que no se encuentran en el conjunto de prueba son columnas tipo "output", valores que se obtienes después del procesado del material, es decir, valores futuros.

## 1.4

Preprocesamiento de datos

Para poder hacer un preprocesamiento adecuado, debemos tomar en cuenta 2 puntos importantes, el primero es que los datos cercanos en el tiempo suelen ser similares, y el segundo es que contamos con parámetros de fecha y hora  para poder definir datos cercanos.

 Buscamos la desviación estandar de 3 muestras de cada columna para corroborar que los datos realmente se acercan entre sí en fechas cercanas, los datos en los df están ordenados por fecha y hora de registro

In [91]:
for _ in range(3):
    rd = np.random.randint(0,22000)
    print(data_full.loc[rd:rd+5, :].drop(["date"], axis=1).std())

final.output.concentrate_ag                   0.322104
final.output.concentrate_pb                   0.337991
final.output.concentrate_sol                  0.154296
final.output.concentrate_au                   0.875786
final.output.recovery                         1.082308
                                                ...   
secondary_cleaner.state.floatbank5_a_level    0.563166
secondary_cleaner.state.floatbank5_b_air      0.686520
secondary_cleaner.state.floatbank5_b_level    0.212719
secondary_cleaner.state.floatbank6_a_air      0.021292
secondary_cleaner.state.floatbank6_a_level    0.342612
Length: 86, dtype: float64
final.output.concentrate_ag                    0.947617
final.output.concentrate_pb                    0.400052
final.output.concentrate_sol                   0.441388
final.output.concentrate_au                    3.075237
final.output.recovery                         12.500623
                                                ...    
secondary_cleaner.state.floatban

Al tener una desviación estandar normalmente no tan alejada en las columnas, sustituiremos los valores vacíos con los valores de el dato anterior más proximo en fecha

In [96]:
data_train.ffill(inplace=True)
data_test.ffill(inplace=True)
data_train.isnull().sum()
data_test.isnull().sum()

date                                          0
primary_cleaner.input.sulfate                 0
primary_cleaner.input.depressant              0
primary_cleaner.input.feed_size               0
primary_cleaner.input.xanthate                0
primary_cleaner.state.floatbank8_a_air        0
primary_cleaner.state.floatbank8_a_level      0
primary_cleaner.state.floatbank8_b_air        0
primary_cleaner.state.floatbank8_b_level      0
primary_cleaner.state.floatbank8_c_air        0
primary_cleaner.state.floatbank8_c_level      0
primary_cleaner.state.floatbank8_d_air        0
primary_cleaner.state.floatbank8_d_level      0
rougher.input.feed_ag                         0
rougher.input.feed_pb                         0
rougher.input.feed_rate                       0
rougher.input.feed_size                       0
rougher.input.feed_sol                        0
rougher.input.feed_au                         0
rougher.input.floatbank10_sulfate             0
rougher.input.floatbank10_xanthate      

## 2.1

Observa cómo cambia la concentracion de metales en función de cada etapa

In [98]:
columnas = pd.Series(data_full.columns)
1 rougher.input.feed_au
3 rougher.output.concentrate_au
5 primary_cleaner.output.concentrate_au
8 final.output.concentrate_au



## 2.1



## 2.1



## 2.1



## 2.1

